In [21]:
import MDAnalysis as mda
import os
import numpy as np

In [22]:
def extract_envbs(protein, lig, radius=5, level='R'):
    """Extract all residues with at least one atom within 5 A from ligand"""
    neighbor = mda.lib.NeighborSearch.AtomNeighborSearch(protein, box=None)
    res_bs = neighbor.search(lig, radius, level=level)
    envbs = res_bs.atoms
    return envbs

def extract_bs(protein, lig, radius=3.5, level='R'):
    """Extract all residues with at least one atom within 3.5 A from ligand"""
    neighbor = mda.lib.NeighborSearch.AtomNeighborSearch(protein, box=None)
    res_bs = neighbor.search(lig, radius, level=level)
    bs = res_bs.atoms
    return bs

def extract_header(pdb):
    """Extract header, title, compnd and hetnam entries"""
    headers = []
    hetnams = []
    with open(pdb, 'r') as fi:
        for line in fi.readlines():
            if line.startswith(('HEADER','TITLE', 'COMPND')):
                headers.append(line)
                if line.startswith('HEADER'):
                    pdbid = line[62:66]
            elif line.startswith('HETNAM'):
                hetnams.append(line)
            elif line.startswith('ATOM'):
                break
                
    return headers, hetnams, pdbid

def write_librabs(pdbpath, ligname, ligid, segid, outdir):
    """Read LIBRAbs and LIBRAenv and create a final LIBRAbindingsite pdb"""
    #Extract info from pdb
    print(f'Extracting info {pdbpath}')
    headers, hetnams, pdbid = extract_header(pdbpath)
    #Join created bs and env in final pdb
    pdbname = f'{pdbid}_{ligname}_{ligid}_{segid}.pdb'
    librapdb = os.path.join(outdir, pdbname)
    outtemp =  os.path.join(outdir, pdbid)
    print(f'Creating {librapdb}')
    with open(f'{librapdb}', 'w') as fo:
        header = ''.join(headers)
        fo.write(header)
        for hetnam in hetnams:
            if ligname in hetnam:    #Write only HETNAM relative to this ligand
                name = hetnam
                fo.write(name)
                break
        #Join bs, 'Enironment\n' and env
        with open(f'{outdir}bs.pdb', 'r') as fi1:
            for line in fi1.readlines():
                if line.startswith('ATOM'):
                    fo.write(line[:-1]+'  \n')
                elif line.startswith('END'):
                    break
            fo.write('Environment\n')
            with open(f'{outdir}env.pdb', 'r') as fi2:
                for line in fi2.readlines():
                    if line.startswith(('ATOM','HETATM')):
                        fo.write(line[:-1]+'  \n')
    #Clean bs and env pdb                   
    os.remove(f'{outdir}bs.pdb')
    os.remove(f'{outdir}env.pdb')

    
    
def extract_librabs(inpdir, outdir):
    """Extract Libra binding sites from pdbs in input folder"""
    for pdb in os.listdir(inpdir):
        print(f'Processing {pdb}')
        pdbpath = os.path.join(inpdir, pdb)
        u = mda.Universe(pdbpath)
        protein = u.select_atoms('protein')
        ligs = u.select_atoms(f'record_type HETATM')
        #Initialize env_old and bs_old for comparisons
        env_old = u.select_atoms('resid -1')
        bs_old = u.select_atoms('resid -1')
        for lig in ligs:
            if lig.resname not in blacklist:    #Check if ligand is in blacklist
                ligname = lig.resname
                ligid = lig.resid
                segid = lig.segid
                lig = u.select_atoms(f'resname {lig.resname} and resid {lig.resid} and segid {lig.segid}')
                #extract bs and env for each ligand
                envbs = extract_envbs(protein, lig)
                bs = extract_bs(protein, lig)
                env = envbs - bs
                env = env + lig
                #Proceed only if bs is not empty
                if any(bs):
                    if any(env_old) and any(bs_old):    #Check if it is a duplicate
                        if np.array_equal(bs.resnames, bs_old.resnames) and np.array_equal(bs.resids, bs_old.resids)\
                        and np.array_equal(env.resnames, env_old.resnames) and np.array_equal(env.resids, env_old.resids):                
                            print('Duplicate found')
                            continue
                        else:
                            bs.write(f'{outdir}bs.pdb', reindex=False)
                            env.write(f'{outdir}env.pdb', reindex=False)
                            write_librabs(pdbpath, ligname, ligid, segid, outdir)
                            bs_old = bs
                            env_old = env
                    else:    #Do it if first run
                        bs.write(f'{outdir}bs.pdb', reindex=False)
                        env.write(f'{outdir}env.pdb', reindex=False)
                        write_librabs(pdbpath, ligname, ligid, segid, outdir)
                        bs_old = bs
                        env_old = env
        
        
        
#Ligands to be removed        
blacklist = ['HOH','NO3','NO2','PO3','PO4','SO4','SO3',\
             'SO2','SUL','MSE','PEG','EPE','IOH','PG4',\
             'DMS','PAS','EDO','TRS','CL1','LDA','BOG',\
             'HEM', 'MOH']

In [23]:
extract_librabs('input/', 'output/')

Processing 9PAP.pdb
Extracting info input/9PAP.pdb
Creating output/9PAP_OCS_25_A.pdb
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Processing 9RUB.pdb
Extracting info input/9RUB.pdb
Creating output/9RUB_RUB_600_A.pdb
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Extracting info input/9RUB.pdb
Creating output/9RUB_MG_500_A.pdb
Extracting info input/9RUB.pdb
Creating output/9RUB_FMT_601_A.pdb
Duplicate found
Duplicate found
Extracting info input/9RUB.pdb
Creating output/9RUB_RUB_700_B.pdb
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicate found
Duplicat

In [24]:
extract_librabs('ZIP_input/', 'ZIP_bs/')

Processing 5TSA.pdb
Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_ZN_401_A.pdb
Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_ZN_403_A.pdb
Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_ZN_404_A.pdb


/home/andrea/miniconda3/envs/compbio/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: CD
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/andrea/miniconda3/envs/compbio/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: SE
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_ZN_405_A.pdb
Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_ZN_406_A.pdb
Extracting info ZIP_input/5TSA.pdb
Creating ZIP_bs/5TSA_CD_407_A.pdb
Processing 5TSB.pdb
Extracting info ZIP_input/5TSB.pdb
Creating ZIP_bs/5TSB_CD_401_A.pdb
Extracting info ZIP_input/5TSB.pdb
Creating ZIP_bs/5TSB_CD_402_A.pdb
Extracting info ZIP_input/5TSB.pdb
Creating ZIP_bs/5TSB_CD_404_A.pdb
Processing 6PGI.pdb
Extracting info ZIP_input/6PGI.pdb
Creating ZIP_bs/6PGI_CD_402_A.pdb
